In [15]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import math
import pandas as pd
import dash_bootstrap_components as dbc
import numpy as np

# Definiere Cluster-Farben
FARBCODIERUNGEN = {'0': "red", '1': "blue", '2': "grey"}

external_stylesheets = ["https://fonts.googleapis.com/css2?family=Lato:wght@400;700&display=swap"]

app = Dash(__name__, external_stylesheets=external_stylesheets)

# Generiere zufällige Daten
def generate_random_data():
    # Generiere deine zufälligen Daten hier
    # Ersetze dies durch deinen tatsächlichen Code, um die Daten zu generieren oder zu laden
    data = {
        'x': [1, 2, 3, 4, 5],
        'y': [10, 20, 30, 40, 50]
    }
    df = pd.DataFrame(data)
    return df

# Generiere zufällige Cluster-Daten
def generate_random_cluster_data():
    # Generiere deine zufälligen Cluster-Daten hier
    # Ersetze dies durch deinen tatsächlichen Code, um die Cluster-Daten zu generieren oder zu laden
    cluster_data = {
        'X': [1, 2, 3, 4, 5],
        'Y': [10, 20, 30, 40, 50],
        'cluster': [0, 1, 0, 1, 2]
    }
    cluster_df = pd.DataFrame(cluster_data)
    return cluster_df

# Generiere zufällige Daten für das Heatmap-Diagramm
def generate_random_data_heatmap():
    x = np.random.randint(0, 10, size=(10, 10))
    return pd.DataFrame(x)

df = generate_random_data()
cluster_df = generate_random_cluster_data()
df_heatmap = generate_random_data_heatmap()

app.layout = html.Div([
    html.Header([html.H1("Dashboard 5")], className="header--title"),
    html.Div([
        dcc.Tabs(id="tabs", children=[
            dcc.Tab(label="Tab1", children=[
                html.Div([
                    html.Div([
                        dcc.Dropdown(
                            options=[{'label': 'Rot', 'value': 'red'}, {'label': 'Grün', 'value': 'green'}, {'label': 'Blau', 'value': 'blue'}],
                            value='red',
                            id='color',
                            multi=False,
                            className="graph_1--dropdown graph-control"
                        ),
                        dcc.Graph(id="graph_1", className="graph_1 graph")
                    ], className="graph-component"),
                    html.Div([
                        dcc.Slider(
                            min=math.floor(df['y'].min()),
                            max=math.ceil(df['y'].max()),
                            id="min_value",
                            className="graph_2--slider graph-control"
                        ),
                        dcc.Graph(id="graph_2", className="graph_2 graph")
                    ], className="graph-component")
                ], className="tab1--main-container")
            ], className="tab1"),
            dcc.Tab(label="Tab2", children=[
                html.Div([
                    html.Div(dcc.Graph(id="graph_3"), className="graph_3"),
                    html.Div(dcc.Graph(id="graph_4"), className="graph_4"),
                    html.Div([
                        dbc.Label("Anzahl der Bins:", html_for="graph_5_nbins"),
                        dcc.Dropdown(
                            options=[{'label': str(i), 'value': str(i)} for i in range(5, 100, 5)],
                            value='40',
                            id='graph_5_nbins',
                            multi=False
                        )
                    ], className="graph_5--bins-dropdown"),
                    html.Div([
                        dbc.Label("Farbe:", html_for="graph_5_color"),
                        dcc.Dropdown(
                            options=[{'label': "Viridis", 'value': "Viridis"}, {'label': "Magma", 'value': "Magma"}, {'label': "Hot", 'value': "Hot"}, {'label': "GnBu", 'value': "GnBu"}, {'label': "Greys", 'value': "Greys"}],
                            value='Hot',
                            id='graph_5_color',
                            multi=False
                        )
                    ], className="graph_5--color-dropdown"),
                    html.Div([
                        dbc.Label("Getrennt nach Cluster:", html_for="graph_5_separated"),
                        dcc.RadioItems(
                            options=[{'label': "Ja", 'value': "Yes"}, {'label': "Nein", 'value': "No"}],
                            value='No',
                            id='graph_5_separated',
                            className="graph_5_separated"
                        )
                    ], className="graph_5--separation-radio"),
                    html.Div([
                        dcc.Graph(id="graph_5", style={'width': '100%', 'height': '100%'}),
                        html.Div(id="cluster-table")
                    ], className="graph_5")
                ], className="tab2--main-container")
            ], className="tab2")
        ], className="tabs-content")
    ], className="container")
])

@app.callback(Output("graph_1", "figure"), Input("color", "value"))
def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    return fig

@app.callback(Output("graph_2", "figure"), Input("min_value", "value"), Input("graph_5", "selectedData"))
def update_graph_2(slider_value, selected_data):
    if selected_data is None:
        filtered_df = df[df['y'] >= slider_value]
    else:
        selected_indices = [point['customdata'] for point in selected_data['points']]
        filtered_df = df[df.index.isin(selected_indices) & (df['y'] >= slider_value)]

    fig = go.Figure(data=go.Bar(x=filtered_df['x'], y=filtered_df['y']))
    return fig

@app.callback(Output("graph_3", "figure"), Output("graph_4", "figure"), Input("graph_5_nbins", "value"), Input("graph_5_color", "value"), Input("graph_5_separated", "value"))
def update_graph_3_4(nbins, color, separated):
    fig3 = px.histogram(df, x="y", nbins=int(nbins), color_discrete_sequence=[color])
    fig4 = px.scatter(cluster_df, x="X", y="Y", color="cluster", color_discrete_map=FARBCODIERUNGEN)
    if separated == "No":
        fig4.update_traces(marker=dict(size=12))
    return fig3, fig4

@app.callback(Output("graph_5", "figure"), Output("cluster-table", "children"), Input("graph_5_separated", "value"), Input("graph_3", "selectedData"))
def update_graph_5(separated, selected_data):
    if separated == "Yes" and selected_data is not None:
        selected_indices = [point['customdata'] for point in selected_data['points']]
        filtered_cluster_df = cluster_df[cluster_df.index.isin(selected_indices)]
        fig5 = px.scatter(filtered_cluster_df, x="X", y="Y", color="cluster", color_discrete_map=FARBCODIERUNGEN)
        fig5.update_traces(marker=dict(size=12))
        cluster_table = html.Table([
            html.Thead(
                html.Tr([
                    html.Th("Cluster"),
                    html.Th("Anzahl der Punkte")
                ])
            ),
            html.Tbody([
                html.Tr([
                    html.Td(cluster),
                    html.Td(len(filtered_cluster_df[filtered_cluster_df["cluster"] == cluster]))
                ]) for cluster in filtered_cluster_df["cluster"].unique()
            ])
        ])
    else:
        fig5 = px.scatter(cluster_df, x="X", y="Y", color="cluster", color_discrete_map=FARBCODIERUNGEN)
        fig5.update_traces(marker=dict(size=12))
        cluster_table = html.Table([
            html.Thead(
                html.Tr([
                    html.Th("Cluster"),
                    html.Th("Anzahl der Punkte")
                ])
            ),
            html.Tbody([
                html.Tr([
                    html.Td(cluster),
                    html.Td(len(cluster_df[cluster_df["cluster"] == cluster]))
                ]) for cluster in cluster_df["cluster"].unique()
            ])
        ])
    return fig5, cluster_table

if __name__ == "__main__":
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


[2023-06-04 21:54:10,795] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\dash\dash.py", line 1249, in dispatch
    ctx.run(
  File "C:\Users\mguen\OneDrive\Desktop\AI\venv\lib\site-packages\dash\_callback.py", line 450, in add_context
    output